In [ ]:
import pandas as pd
from math import *
import numpy as np

# sentiment analysis

In [ ]:
from textblob import Blobber
#a library that support french language 
from textblob_fr import PatternTagger, PatternAnalyzer

def sentiment(rev):
    blob = TextBlob(rev)
    #print(blob.sentiment)
    if blob.sentiment.polarity > 0:
        sent = "pos"
    else:
        sent = "neg"
    return sent
# for french reviews 
def sentiment_FR(rev):
    tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
    blob = tb(rev)
    if blob.sentiment[0] > 0:
        sent = "pos"
    else:
        sent = "neg"
    return sent

# find country

In [ ]:
from geopy.geocoders import Nominatim

def findCountry(pos):
    # Initialize Nominatim API
    print(pos)
    geolocator = Nominatim(user_agent="MyApp")
    try:
        location = geolocator.geocode(pos,timeout=10000)
        lat = location.latitude
        lon = location.longitude
        loc = geolocator.reverse(str(lat)+','+str(lon))
        adr = loc.raw['address']
        country = adr.get('country')
        if country.find('/') != -1:
            country = country[0:country.index('/')]
        country = TextBlob(country)
        try:
            country = country.translate(to='en')
            country = str(country).rstrip()
        except Exception:
            country = str(country).rstrip()
    except AttributeError:
        return "ALL"
    return country

In [ ]:
dataFr = pd.read_csv("finalDataFR.csv", header=0)
dataEn = pd.read_csv("finalData.csv", header=0)

# Reviews classification 

In [ ]:
#sentiment analysis for frensh reviews

# =======following commands are commented ; the task already done========

# dataFr['type_rev'] = dataFr['review'].apply(lambda x : sentiment_FR(x))
# dataEn['type_rev'] = dataEn['review'].apply(lambda x : sentiment(x))


# booking reviews

In [ ]:
bookingData = pd.read_csv("../final/bookingRevs.csv", sep=";", header = 0)

In [ ]:
bookingData

In [ ]:
bookingData.rename(columns={"ville":"city"}, inplace=True)
bookingData.rename(columns={"reviewer_rate":"rate"}, inplace=True)

## rescale booking review

In [ ]:
#initial values are scaled on 10 , we want to rescale them to 5
bookingData.rate = bookingData.rate.apply(lambda x : floor(round(x/2)))
bookingData.rate = bookingData.rate.apply(lambda x : 1 if (x == 0) else x )

# Merging dataFrames

In [ ]:
bookingData

In [ ]:
dataEn

In [ ]:
bookingData = bookingData[['hotel_name', 'city','rate', 'review','reviewer_name','reviewer_country','stay_date','type_rev']]

In [ ]:
bookingData

In [ ]:
df = dataEn.append(dataFr.append(bookingData, ignore_index=True), ignore_index=True)

# find country

to avoid blocking while using geopy, we try to minimize the request number, so that we use a list with unique countries found

In [ ]:
#==============commands below are already executed
#=================================================
#temp = df['reviewer_country'].unique()
#Cnt = []
#for c in temp:
#    Cnt.append(findCountry(c))
#df['reviewer_country'] = df['reviewer_country'].apply(lambda x: "ALL" if pd.isna(x) else Cnt[int(np.where(temp==x)[0])] )

In [ ]:
df

# Transform

## unify city and hotel names

In [ ]:
df.city.unique()

In [ ]:
df['city'] = df['city'].apply(lambda x : "Tangier" if x == "Tanger (Tanja)" else x )

In [ ]:
hotel_corr = pd.read_csv('correspondance_Hotels.csv', sep=",", header = 0)

In [ ]:
hotel_corr

In [ ]:
df.hotel_name = df.hotel_name.str.strip()

In [ ]:
for i in range(hotel_corr.shape[0]):
    df.hotel_name = df.hotel_name.str.replace(hotel_corr.hoteltrip[i],hotel_corr.hotelbook[i], regex=False)

# create dimension tables

In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df

## dimension date

In [ ]:
date = df.stay_date.unique()
date = pd.DataFrame(date)
date.reset_index(inplace=True)
date.rename(columns={0 : 'stay_date', 'index' : 'id_date'}, inplace = True)
date['month'] = date['stay_date'].apply(lambda x : str(x).split(" ")[0])
date['year'] = date['stay_date'].apply(lambda x : str(x).split(" ")[1])
date

In [ ]:
#final dataframe
dim_date = date

## dimension hotel

In [ ]:
hotel_info = pd.read_csv("hotel_info.csv", sep=";", header = 0)

In [ ]:
hotel_info

In [ ]:
for i in range(hotel_corr.shape[0]):
    hotel_info.hotel_name = hotel_info.hotel_name.str.replace(hotel_corr.hoteltrip[i],hotel_corr.hotelbook[i], regex=False)

In [ ]:
hotel_info.reset_index(inplace=True)
hotel_info.rename(columns={'index' : 'id_hotel'}, inplace = True)

In [ ]:
#final dataframe
dim_hotel = hotel_info

# twitter

In [ ]:
df_temp = df

In [ ]:
tweets = pd.read_csv("Tweets_Rev_Final.csv", sep=";", header = 0)

In [ ]:
tweets.dropna(inplace=True)

In [ ]:
tweets.hotel_name = tweets.hotel_name.str.strip()

In [ ]:
rev = df_temp.reviewer_name.unique()
t_rev = tweets.reviewer_username.unique()
reviewers = np.concatenate([rev,t_rev])

In [ ]:
rev_name = pd.DataFrame(np.unique(reviewers), columns=['reviewer_name'])
#for reviewer country
rev_country = pd.DataFrame(df.reviewer_country.unique(), columns=['reviewer_country'])

rev_name.reset_index(inplace=True)

rev_name.rename(columns={"index": "id_reviewer"}, inplace=True)

rev_country.reset_index(inplace=True)

rev_country.rename(columns={"index": "id_country"}, inplace=True)

In [ ]:
tweets.rename(columns={"reviewer_username":"reviewer_name"}, inplace=True)
tweets.rename(columns={"reviewer_location":"reviewer_country"}, inplace=True)

# merging ....

In [ ]:
df_temp = pd.merge(df_temp, rev_name, on='reviewer_name')

In [ ]:
tweets_temp = pd.merge(tweets, rev_name, on="reviewer_name")

In [ ]:
tweets_temp

In [ ]:
tweets_temp = pd.merge(tweets_temp, rev_country, on="reviewer_country")
df_temp = pd.merge(df_temp, rev_country, on='reviewer_country')

In [ ]:
tweets_temp

In [ ]:
tweets_temp = pd.merge(tweets_temp, hotel_info, on="hotel_name")
df_temp = pd.merge(df_temp, hotel_info, on='hotel_name')

In [ ]:
tweets_temp

In [ ]:
tweets_temp = pd.merge(tweets_temp, date, right_on="stay_date", left_on='date_of_review')
df_temp = pd.merge(df_temp, date, on='stay_date')

## dimension reviewer

In [ ]:
reviewer_tw = tweets_temp[['id_reviewer', 'reviewer_name','id_country']]
reviewer_tw.drop_duplicates(inplace = True)
reviewer_rv = df_temp[['id_reviewer', 'reviewer_name','id_country']]
reviewer_rv.drop_duplicates(inplace = True)
dim_reviewer = reviewer_rv.append(reviewer_tw, ignore_index=True)

In [ ]:
#final dataframe
dim_country = rev_country

# fact tables

In [ ]:
tweets_temp

In [ ]:
df_temp.rename(columns={'type_rev' : 'rev_class'}, inplace=True)

In [ ]:
tweets_temp.rename(columns={'classification' : 'rev_class'}, inplace=True)

### fact_reviews

In [ ]:
fact_review = df_temp[['id_hotel', 'id_date', 'id_reviewer', 'rate', 'review','rev_class']]

### fact_tweets

In [ ]:
fact_tweets = tweets_temp[['id_hotel', 'id_date', 'id_reviewer','likes','review','rev_class' ]]

In [ ]:
fact_tweets

# Create output files

In [195]:
fact_review.rev_class.unique()

array(['pos', 'neg'], dtype=object)

In [196]:
#avoid delimiter problems
fact_review.review = fact_review.review.str.replace('"','')
fact_tweets.review = fact_tweets.review.str.replace('"','')

fact_review.review = fact_review.review.str.replace(";",".")
fact_tweets.review = fact_tweets.review.str.replace(";",".")

C:\Users\win00\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [197]:
dim_country.to_csv("output/dim_country.csv", sep=";",index= False)
dim_date.to_csv("output/dim_date.csv", sep=";",index= False)
dim_hotel.to_csv("output/dim_hotel.csv", sep=";",index= False)
dim_reviewer.to_csv("output/dim_reviewer.csv", sep=";",index= False)
fact_review.to_csv("output/fact_review.csv", sep=";",index= False)
fact_tweets.to_csv("output/fact_tweets.csv", sep=";",index= False)